# Union-Find Data Structure and Island Counting

#### The code below defines a Union-Find data structure (also known as Disjoint Set Union, DSU) that helps manage groups of elements (or sets) efficiently. It supports two main operations: union, which merges two sets, and find, which identifies the root of a set containing a specific element.

#### Additionally, the code implements a function, num_islands, that keeps track of the number of islands formed in a grid as land is added. An island is defined as a group of adjacent land cells (horizontally or vertically).

In [ ]:
# no init

In [ ]:
# gray and black are used to track the states of nodes

In [ ]:
# Constants representing the colors for the Union-Find states
GRAY, BLACK = 0, 1

def top_sort_recursive(graph):
    """ 
    Perform a topological sort using depth-first search (DFS).
    Time complexity is O(V + E), where V is the number of vertices and E is the number of edges.
    Space complexity is O(V).
    """
    order = []  # List to store the topological order
    enter = set(graph)  # Set of all nodes to track unvisited nodes
    state = {}  # Dictionary to track the state of each node

    def dfs(node):
        """ Helper function to perform DFS on the graph. """
        state[node] = GRAY  # Mark the node as being visited
        for k in graph.get(node, ()):  # Iterate through adjacent nodes
            sk = state.get(k, None)  # Get the state of the adjacent node
            if sk == GRAY:  # Check for cycles
                raise ValueError("cycle")
            if sk == BLACK:  # Skip already processed nodes
                continue
            enter.discard(k)  # Remove node from unvisited set
            dfs(k)  # Recursively visit the adjacent node
        order.append(node)  # Append the current node to the order
        state[node] = BLACK  # Mark the node as fully processed

    while enter:  # Continue until all nodes are processed
        dfs(enter.pop())  # Start DFS with an unvisited node
    return order  # Return the topological order

def top_sort(graph):
    """ 
    Perform a topological sort iteratively.
    Time complexity is O(V + E).
    Space complexity is O(V).
    """
    order = []  # List to store the topological order
    enter = set(graph)  # Set of all nodes to track unvisited nodes
    state = {}  # Dictionary to track the state of each node

    def is_ready(node):
        """ Check if all parents of the current node are processed. """
        lst = graph.get(node, ())
        if len(lst) == 0:  # If there are no adjacent nodes
            return True
        for k in lst:
            sk = state.get(k, None)  # Get the state of the adjacent node
            if sk == GRAY:  # Check for cycles
                raise ValueError("cycle")
            if sk != BLACK:  # If a parent is not fully processed
                return False
        return True  # All parents are processed

    while enter:
        node = enter.pop()  # Get an unvisited node
        stack = []  # Stack for iterative DFS
        while True:
            state[node] = GRAY  # Mark the node as being visited
            stack.append(node)  # Add the node to the stack
            for k in graph.get(node, ()):  # Iterate through adjacent nodes
                sk = state.get(k, None)  # Get the state of the adjacent node
                if sk == GRAY:  # Check for cycles
                    raise ValueError("cycle")
                if sk == BLACK:  # Skip already processed nodes
                    continue
                enter.discard(k)  # Remove node from unvisited set
                stack.append(k)  # Add adjacent node to the stack
            while stack and is_ready(stack[-1]):
                node = stack.pop()  # Process the node if it's ready
                order.append(node)  # Add the node to the order
                state[node] = BLACK  # Mark the node as fully processed
            if len(stack) == 0:  # If stack is empty, break
                break
            node = stack.pop()  # Get the next node from the stack
        
    return order  # Return the topological order

class Union:
    """
    A Union-Find data structure to manage disjoint sets.

    Example usage:
    Initially, sets are separated:
    {1} {2} {3} {4}
    After unite(1, 3):
    {1, 3} {2} {4}
    """

    def __init__(self):
        self.parents = {}  # Dictionary to track the parent of each element
        self.size = {}     # Dictionary to track the size of each set
        self.count = 0     # Count of disjoint sets

    def add(self, element):
        """ 
        Add a new element as its own set.
        """
        self.parents[element] = element  # Initialize the parent of the element to itself
        self.size[element] = 1  # Set size of the new set to 1
        self.count += 1  # Increment the count of disjoint sets

    def root(self, element):
        """ 
        Find the root element of the set containing the specified element.
        Path compression is applied for efficiency.
        """
        while element != self.parents[element]:  # Traverse up to the root
            self.parents[element] = self.parents[self.parents[element]]  # Path compression
            element = self.parents[element]
        return element  # Return the root

    def unite(self, element1, element2):
        """ 
        Unite the sets containing the two specified elements.
        """
        root1, root2 = self.root(element1), self.root(element2)  # Find roots of both elements
        if root1 == root2:  # If they are already in the same set, do nothing
            return
        # Union by size: attach smaller tree under larger tree
        if self.size[root1] > self.size[root2]:
            root1, root2 = root2, root1  # Ensure root1 is the smaller
        self.parents[root1] = root2  # Link root1 to root2
        self.size[root2] += self.size[root1]  # Update the size of the new root
        self.count -= 1  # Decrease the number of disjoint sets

def num_islands(positions):
    """
    Count the number of islands after each addLand operation.
    An island is formed by connecting adjacent lands.

    Example:
    Given positions = [[0,0], [0,1], [1,2], [2,1]].
    """
    ans = []  # List to keep track of the number of islands
    islands = Union()  # Create a Union-Find structure for the positions
    for position in map(tuple, positions):  # Iterate through the positions
        islands.add(position)  # Add the position as land
        # Check adjacent positions (up, down, left, right)
        for delta in (0, 1), (0, -1), (1, 0), (-1, 0):
            adjacent = (position[0] + delta[0], position[1] + delta[1])  # Calculate adjacent position
            if adjacent in islands.parents:  # If adjacent position is land
                islands.unite(position, adjacent)  # Unite the current land with the adjacent land
        ans += [islands.count]  # Append the current number of islands to the result
    return ans  # Return the list of island counts
